# Simulation of the polarized neutron diffraction pattern
This notebook shows how we can create a sample (phase) from atoms and calculate diffraction profiles. The results are optimized to match the experimental data.

In [ ]:
# esyScience, technique-independent
import numpy as np
from easyscience.fitting.fitter import Fitter
# esyScience, diffraction
from easyDiffractionLib import Site, Phase, Phases
from easycrystallography.Components.Susceptibility import MagneticSusceptibility
from easyDiffractionLib.Jobs import PolPowder1DCW
from easyDiffractionLib.elements.Backgrounds.Point import PointBackground, BackgroundPoint
import xarray as xr

# Vizualization
import py3Dmol
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
try:
    import hvplot.xarray  # noqa
except ImportError:
    ! pip install hvplot
    import hvplot.xarray  # noqa
from hvplot import hvPlot
hvplot.extension('bokeh')
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column
from bokeh.plotting import figure
output_notebook()
from bokeh.palettes import Spectral6

In [ ]:
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300
opts = dict(width=FIGURE_WIDTH, height=FIGURE_HEIGHT, min_border=0)
ds = xr.Dataset()

## Fe3O4 Sample

In this example we use constructors to build a sample, starting with atoms, space-group and lattice parameters.

####  Create  atoms using `Site` methods

In [ ]:
Fe3A = Site(label="Fe3A",
            specie="Fe3+",
            fract_x=0.125,
            fract_y=0.125,
            fract_z=0.125,
            msp=MagneticSusceptibility('Cani', chi_11=-4.0380, chi_22=-4.0380, chi_33=-4.0380))
Fe3B = Site(label="Fe3B",
            specie="Fe3+",
            fract_x=0.5,
            fract_y=0.5,
            fract_z=0.5,
            msp=MagneticSusceptibility('Cani', chi_11=3.9318, chi_22=3.9318, chi_33=3.9318))
O = Site(label="O",
        specie="O2-",
        fract_x=0.25521,
        fract_y=0.25521,
        fract_z=0.25521)

#### Creating a `Phase`

We create a phase and set space-group and previously created atoms. Space-group information can also be set by calling the `SpaceGroup` constructor.

In [ ]:
phase = Phase(name="Fe3O4")
phase.spacegroup.space_group_HM_name = "F d -3 m:2"
phase.add_atom(Fe3A)
phase.add_atom(Fe3B)
phase.add_atom(O)

The unit-cell parameters can be set by modifying the cell attributes.

In [ ]:
phase.cell.a = 8.56212

Because the unit-cell is a cubic, the lattice parameters are the same. This should be automatically applied.

In [ ]:
print(phase.cell.length_b)
print(phase.cell.length_c)

#### Visualise the structure

Using `py3Dmol` we can visualise the phases structure.

In [ ]:
viewer = py3Dmol.view()
viewer.addModel(phase.cif,'cif',{'doAssembly':True,'duplicateAssemblyAtoms':True,'normalizeAssembly':True})
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,2)
viewer.zoomTo()

#### Create Phases object

The created phase is wrapped in a phases object.

In [ ]:
phases = Phases()
phases.append(phase)

## Simulating the polarized diffraction pattern

The easiest way of simulating a diffraction pattern is to use the `PolPowder1DCW` class. In this case we call the job `Fe3O4_test` and modify the experimental resolution parameters

In [ ]:
j1 = PolPowder1DCW('Fe3O4_test', ds, phases=phases)
parameters = j1.parameters
parameters.resolution_u = 0.447
parameters.resolution_v = -0.4252
parameters.resolution_w = 0.3864
parameters.resolution_x = 0.0
parameters.resolution_y = 0.0

There are also pattern parameters, we can set them.

In [ ]:
pattern = j1.pattern
pattern.zero_shift = 0.0
pattern.scale = 10.0
pattern.field = 4.0

By default the job interface uses the `CrysPy` calculator. This is a wrapper around the `CrysPy` library. It can be verified by calling the `interface` property.

In [ ]:
calculator = j1.interface

In [ ]:
print(f"Current calculator engine: {calculator.current_interface_name}")

#### Calculating the profiles
We create a simulation range and calculate the diffraction pattern for three cases; Spin up, Spin down and the Spin difference.

In [ ]:
x_data = np.linspace(20, 120, 800)
a = j1.create_simulation(x_data, 'up', pol_fn=lambda up, down: up)
b = j1.create_simulation(x_data, 'down', pol_fn=lambda up, down: down)
c = j1.create_simulation(x_data, 'diff', pol_fn=lambda up, down: up - down)

These profiles can be plotted

In [ ]:
p1 = figure(**opts, title='Fe3O4 Polarization')
p1.line(x_data, np.array(ds['sim_Fe3O4_test_up']), legend='Spin Up', line_width=2, color=Spectral6[0])
p1.line(x_data, np.array(ds['sim_Fe3O4_test_down']),  legend='Spin Down', line_width=2, color=Spectral6[1])
p1.line(x_data, np.array(ds['sim_Fe3O4_test_diff']),  legend='Difference', line_width=2, color=Spectral6[2])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p1.xaxis.axis_label = '2theta'
show(p1)

# Fitting a polarized powder profile

In this example we load a sample from a cif file and modify some parameters to be more physically meaningful.

Load the phase from a cif file:

In [ ]:
p2 = Phases.from_cif_file('structure_polarized.cif')
p2[0].cell.length_a.fixed = False # Optimum value of the lattice parameter `a` is: 10.26594(88)

Create a new job and set the phase and vaguely acceptable experimental parameters.

In [ ]:
j2 = PolPowder1DCW('pol2', ds, phases=p2)
j2.phases[0].scale = 0.025
j2.phases[0].scale.fixed = False
j2.parameters.wavelength = 0.84
j2.parameters.resolution_u = 15
j2.parameters.resolution_u.fixed = False
j2.parameters.resolution_v = -2.5
j2.parameters.resolution_v.fixed = False
j2.parameters.resolution_w = 0.375
j2.parameters.resolution_w.fixed = False
j2.parameters.resolution_x = 0.0
j2.parameters.resolution_y = 0.225
j2.parameters.resolution_y.fixed = False
j2.pattern.zero_shift = 0.35
j2.pattern.zero_shift.fixed = False
j2.pattern.beam.polarization = 0.6909

Perform an initial simulation to verify that the parameters make a meaningful pattern.

In [ ]:
j2.create_simulation(x_data, 'up', pol_fn=lambda up, down: up)
j2.create_simulation(x_data, 'down', pol_fn=lambda up, down: down)
j2.create_simulation(x_data, 'difference', pol_fn=lambda up, down: up - down)


p1 = figure(**opts, title='Ho2Ti2O7 Polarization')
x_data = np.array(ds['sim_pol2_tth'])
p1.line(x_data, np.array(ds['sim_pol2_up']), legend='Spin Up', line_width=2, color=Spectral6[0])
p1.line(x_data, np.array(ds['sim_pol2_down']),  legend='Spin Down', line_width=2, color=Spectral6[1])
p1.line(x_data, np.array(ds['sim_pol2_difference']),  legend='Difference', line_width=2, color=Spectral6[2])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p1.xaxis.axis_label = '2theta'
show(p1)

### Load experimental data

The data file contains the following columns; `tth`, `up`, `d_up`, `down`, `d_down`. The `d_` prefixes denote the uncertainty of the data. The data can be loaded with the `add_experimnent` method.

In [ ]:
j2.add_experiment('pol_exp','experiment_polarized.xye')

This experimental data can then be used for simulation.

In [ ]:
j2.simulate_experiment('pol_exp', 'up', pol_fn=lambda up, down: up)

p1 = figure(**opts, title='Ho2Ti2O7 Polarization')
x_data = np.array(ds['pol2_pol_exp_tth'])
p1.line(x_data, np.array(ds['pol2_pol_exp_I0']), legend='Experimental Spin Up', line_width=2, color=Spectral6[0])
p1.line(x_data, np.array(ds['sim_pol2_pol2_pol_expup']),  legend='Simulated Spin Up', line_width=2, color=Spectral6[-1])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p1.xaxis.axis_label = '2theta'
show(p1)

#### Adding a background
The simulation is missing a background. We can add a background to the simulation and allow the points to vary in the optimization.

In [ ]:
bkg = PointBackground(linked_experiment='pol2')
bkg.append(BackgroundPoint.from_pars(5.0, 480.0))
bkg.append(BackgroundPoint.from_pars(10.0, 420.0))
bkg.append(BackgroundPoint.from_pars(15.0, 360.0))
bkg.append(BackgroundPoint.from_pars(20.0, 360.0))
bkg.append(BackgroundPoint.from_pars(25.0, 325.0))
bkg.append(BackgroundPoint.from_pars(30.0, 325.0))
bkg.append(BackgroundPoint.from_pars(35.0, 325.0))
bkg.append(BackgroundPoint.from_pars(40.0, 250.0))
bkg.append(BackgroundPoint.from_pars(45.0, 275.0))
bkg.append(BackgroundPoint.from_pars(50.0, 245.0))
bkg.append(BackgroundPoint.from_pars(55.0, 270.0))
bkg.append(BackgroundPoint.from_pars(60.0, 215.0))
bkg.append(BackgroundPoint.from_pars(65.0, 260.0))
bkg.append(BackgroundPoint.from_pars(70.0, 250.0))
bkg.append(BackgroundPoint.from_pars(75.0, 230.0))
bkg.append(BackgroundPoint.from_pars(80.0, 225.0))
bkg.append(BackgroundPoint.from_pars(85.0, 250.0))
j2.set_background(bkg)

for point in bkg:
    point.y.fixed = False

Perform another simulation to verify that the background is now included in the simulation.

In [ ]:
j2.simulate_experiment('pol_exp', 'up', pol_fn=lambda up, down: up)

p1 = figure(**opts, title='Ho2Ti2O7 Polarization')
x_data = np.array(ds['pol2_pol_exp_tth'])
p1.line(x_data, np.array(ds['pol2_pol_exp_I0']), legend='Experimental Spin Up', line_width=2, color=Spectral6[0])
p1.line(x_data, np.array(ds['sim_pol2_pol2_pol_expup']),  legend='Simulated Spin Up', line_width=2, color=Spectral6[-1])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p1.xaxis.axis_label = '2theta'
show(p1)

### Optimization

We can now optimize the model. To do this we need to specify what pattern components to optimize. In this case we want to optimize both the `Spin Up + spin Down` and `Spin Up - spin Down` components.  Luckily there are helper functions to do this.

In [ ]:
# Experimental data
xx = np.array(ds['pol2_pol_exp_tth'])
ups = np.array(ds['pol2_pol_exp_I0'])
downs = np.array(ds['pol2_pol_exp_I1'])

In [ ]:
targets = [lambda u, d: u+d , lambda u, d: u-d]
x_ = xx
y_ = ups+downs
x_, y_, f = j2.interface().generate_pol_fit_func(xx, ups, downs, targets)


The optimization can now be performed.

In [ ]:
fit = Fitter(j2, f)
res = fit.fit(x_, y_)

It is easier to visualize the results of the optimization.

In [ ]:
p1 = figure(**opts, title='Polarization, U + D')
p1.scatter(x_[0::2], res.y_obs[0::2], legend='U + D', color=Spectral6[0])
p1.line(x_[0::2], res.y_calc[0::2],  legend='Sim', line_width=2, color=Spectral6[-1])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p11 = figure(width=FIGURE_WIDTH, height=int(FIGURE_HEIGHT/2), min_border=0, title='Difference')
p11.line(x_[0::2], res.y_obs[0::2] - res.y_calc[0::2], line_width=2, color=Spectral6[1])
p11.xaxis.axis_label = '2theta'
show(column(p1, p11))

In [ ]:
p2 = figure(**opts, title='Polarization, U - D')
p2.scatter(x_[1::2], res.y_obs[1::2], legend='U - D', color=Spectral6[0])
p2.line(x_[1::2], res.y_calc[1::2],  legend='Sim', line_width=2, color=Spectral6[-1])
p2.yaxis.axis_label = 'Intensity'
p2.legend.location = 'top_right'
p22 = figure(width=FIGURE_WIDTH, height=int(FIGURE_HEIGHT/2), min_border=0, title='Difference')
p22.line(x_[1::2], res.y_obs[1::2] - res.y_calc[1::2], line_width=2, color=Spectral6[1])
p22.xaxis.axis_label = '2theta'
show(column(p2, p22))

The optimization results are shown below.

In [ ]:
parameters = j2.get_fit_parameters()
for parameter in parameters:
    print(parameter)

#### Varying Magnetic Susceptibility

It looks like the magnetic susceptibility also needs to be optimized. We need to apply some constraints for optimization.

In [ ]:
from easyscience.Fitting.Constraints import ObjConstraint
c1 = ObjConstraint(j2.phases[0].atoms[0].msp.chi_22, '', j2.phases[0].atoms[0].msp.chi_11)
c2 = ObjConstraint(j2.phases[0].atoms[0].msp.chi_33, '', j2.phases[0].atoms[0].msp.chi_11)
c3 = ObjConstraint(j2.phases[0].atoms[0].msp.chi_13, '', j2.phases[0].atoms[0].msp.chi_12)
c4 = ObjConstraint(j2.phases[0].atoms[0].msp.chi_23, '', j2.phases[0].atoms[0].msp.chi_12)

j2.phases[0].atoms[0].msp.chi_11.user_constraints['chi_22'] = c1
j2.phases[0].atoms[0].msp.chi_11.user_constraints['chi_33'] = c2
j2.phases[0].atoms[0].msp.chi_11.fixed = False
j2.phases[0].atoms[0].msp.chi_12.user_constraints['chi_13'] = c3
j2.phases[0].atoms[0].msp.chi_12.user_constraints['chi_23'] = c4
j2.phases[0].atoms[0].msp.chi_12.fixed = False

In [ ]:
res = fit.fit(x_, y_)

Visualize the results of the optimization.

In [ ]:
p1 = figure(**opts, title='Polarization, U + D')
p1.scatter(x_[0::2], res.y_obs[0::2], legend='U + D', color=Spectral6[0])
p1.line(x_[0::2], res.y_calc[0::2], legend='Sim', line_width=2, color=Spectral6[-1])
p1.yaxis.axis_label = 'Intensity'
p1.legend.location = 'top_right'
p11 = figure(width=FIGURE_WIDTH, height=int(FIGURE_HEIGHT / 2), min_border=0, title='Difference')
p11.line(x_[0::2], res.y_obs[0::2] - res.y_calc[0::2], line_width=2, color=Spectral6[1])
p11.xaxis.axis_label = '2theta'
show(column(p1, p11))

In [ ]:
p2 = figure(**opts, title='Polarization, U - D')
p2.scatter(x_[1::2], res.y_obs[1::2], legend='U - D', color=Spectral6[0])
p2.line(x_[1::2], res.y_calc[1::2], legend='Sim', line_width=2, color=Spectral6[-1])
p2.yaxis.axis_label = 'Intensity'
p2.legend.location = 'top_right'
p22 = figure(width=FIGURE_WIDTH, height=int(FIGURE_HEIGHT / 2), min_border=0, title='Difference')
p22.line(x_[1::2], res.y_obs[1::2] - res.y_calc[1::2], line_width=2, color=Spectral6[1])
p22.xaxis.axis_label = '2theta'
show(column(p2, p22))

The optimization results are shown below.

In [ ]:
parameters = j2.get_fit_parameters()
for parameter in parameters:
    print(parameter)